In [1]:
!ls

sample_data  spam.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Model

from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('spam.csv', encoding='latin-1')

In [5]:
# convert labels to binary numbers
df['class'] = df['v1'].map({'ham': 0, 'spam': 1})
Y = df['class'].values
df_train, df_test, Ytrain, Ytest = train_test_split(df['v2'], Y, test_size=0.2)

In [6]:
# Convert string into vector of integers
MAX_VOCAB_SIZE = 20000
MAX_SEQUENCE_SIZE = 160

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

# pad the sequences so each vector in the matrix has same length
data_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_SIZE)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_SIZE)

print('Shape of data train tensor:', data_train.shape)
print('Shape of data test tensor:', data_test.shape)

Shape of data train tensor: (4457, 160)
Shape of data test tensor: (1115, 160)


In [7]:
# Create the model

x = Input(shape=(data_train.shape[1],))
y = Embedding(len(tokenizer.word_index) + 1, 20)(x)
y = LSTM(15, return_sequences=True)(y)
y = GlobalMaxPooling1D()(y)
y = Dense(1, activation='sigmoid')(y)

model = Model(inputs=x, outputs=y)

In [8]:
# Compile and fit
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

fitted_model = model.fit(
  x=data_train,
  y=Ytrain,
  epochs=30,
  validation_split=0.2,
)

Epoch 1/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 16s 110ms/step - accuracy: 0.8556 - loss: 0.4881 - val_accuracy: 0.8666 - val_loss: 0.2831
Epoch 2/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 17s 80ms/step - accuracy: 0.8977 - loss: 0.2150 - val_accuracy: 0.9821 - val_loss: 0.0870
Epoch 3/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9882 - loss: 0.0677 - val_accuracy: 0.9854 - val_loss: 0.0587
Epoch 4/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9938 - loss: 0.0367 - val_accuracy: 0.9832 - val_loss: 0.0661
Epoch 5/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 11s 74ms/step - accuracy: 0.9969 - loss: 0.0245 - val_accuracy: 0.9877 - val_loss: 0.0454
Epoch 6/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9966 - loss: 0.0146 - val_accuracy: 0.9865 - val_loss: 0.0437
Epoch 7/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.9993 - loss: 0.0105 - val_accuracy: 0.9877 - val_loss: 0.0433
Epoch 8/30
112/112 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.9996 - loss: 0.0077 - v

In [9]:
Ypreds = (model.predict(data_test) > 0.5).astype("int32")
print('Model Accuracy score:', metrics.accuracy_score(Ytest, Ypreds))

35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step
Model Accuracy score: 0.9829596412556054
